In [6]:
from database_cloudant import CloudantDB
from cloudant.design_document import DesignDocument
from cloudant.view import View
import sys



#db1 = CloudantDB("tweets_keywords")    # tweets in the specified location with keywords 
#db2 = CloudantDB("tweets_no_keywords") # tweets in the sepcified locations without keyowrds
#db3 = CloudantDB("tweets_raw")         # raw tweet data in the specified locations
#print(len(db1.get_data()))
#print(db2.get_data(n=10000))
#print(len(db3.get_data(n=10)))


def sentiment_zone(url):
    db1 = CloudantDB("tweets_keywords",url=url)    # tweets in the specified location with keywords 
    db2 = CloudantDB("tweets_no_keywords",url=url) # tweets in the sepcified locations without keyowrds
    # get sentiment stats by zone
    ddoc = DesignDocument(db2.curDB, '_design/ddoc001')
    ddoc.fetch()
    view = View(ddoc, 'view_zone')
    summary = []
    for row in view(group=True)['rows']:
        summary.append(row)
    
    db1.close()
    db2.close()
    return summary

    
def dataofday(date,url):    
    # get all data from a specific day
    db1 = CloudantDB("tweets_keywords",url=url)    # tweets in the specified location with keywords 
    db2 = CloudantDB("tweets_no_keywords",url=url) # tweets in the sepcified locations without keyowrds
    n = 100  # number of tweets need to retrieve
    ddoc = DesignDocument(db2.curDB, '_design/ddoc001')
    ddoc.fetch()
    view = View(ddoc,"view_time")
    data = []
    for row in view(group=False,limit=n,reduce=False,key=date,include_docs=True)["rows"]:
        data.append(row)
    
    db1.close()
    db2.close()
    return data

    
    
def sentiment_day(date,url):
    db1 = CloudantDB("tweets_keywords",url=url)    # tweets in the specified location with keywords 
    db2 = CloudantDB("tweets_no_keywords",url=url) # tweets in the sepcified locations without keyowrds
    # get sentiment stats by day
    ddoc = DesignDocument(db2.curDB, '_design/ddoc001')
    ddoc.fetch()
    view = View(ddoc,"view_time")
    sentiment_day = []
    for row in view(group=True,reduce=True)["rows"]:
        sentiment_day.append(row)
    db1.close()
    db2.close()
    return sentiment_day

    


if __name__ == "main":
    ipaddress=sys.argv[1]
    url="http://"+ipaddress
    sentimental_zone=sentiment_zone(ipaddress)
    print(sentimental_zone)
    date = "2021-01-01"
    data=dataofday(date)

    sentiofday=sentiment_day(date,ipaddress)
    print(sentiofday)

In [5]:
!pip install cloudant

  Created wheel for cloudant: filename=cloudant-2.14.0-py3-none-any.whl size=75191 sha256=59a01b3258f8d3f67b1a87b73142000cf62307c46670b8413b15c1d6cda7e33c
  Stored in directory: c:\users\maoya\appdata\local\pip\cache\wheels\ee\5f\f5\35de12a247e01110c89054ea38ad435931eb9989e9dbf1de5f
Successfully built cloudant
